# Markov Chain Monte Carlo optimization

In this assignment, we're going to walk through an example of how to use the [Markov Chain Monte Carlo](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo) (MCMC) technique to find an estimate of the best fit of a curve to data.  We're first going to create a fake dataset with errors introduced that we can control, and will then use MCMC to both find a best-fit solution and the distribution of probabilities around that solution.

## First step: Generating data

We're going to create data using a parabola, which has a simple functional form $f(x) = Ax^2 + B$, and add noise to it.  You can control the constants ```A``` and ```B``` as well as the noise added, which is Gaussian with zero mean and variance ```sigma```.  Examine the code below to see exactly what happens.

**For the moment,** leave the parameter values alone and just look through the data.  You will come back and change this later!

In [ ]:
'''
---------------- USER SETS THESE! ----------------
 A_user = parabola width
 B_user = y-axis intercept
 sigma = error added to the noise
 data_points = number of data points we want to use
 estimated_error = what we assume the error to be in our MCMC fitting - this does NOT 
                   have to be the same as sigma, if we want to see what happens when
                   we make poor assumptions about our actual error
'''

A_user = 0.75
B_user = -3.0
sigma = 2.0
data_points=10
estimated_error = sigma

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as npr

# You CAN set the seed so we get predictable (i.e., reproducible) results.
# To do so, just uncomment the "npr.seed" line right below this comment:
# npr.seed(8675309)

def my_model(A,B,sigma=0.5,data_points=10):
    '''
    Creates a simple set of data points in a parabola that has 
    noise added in the y-direction.  The user can control the parabola
    properties (A and B) as well as the noise level and number of data
    points.  Returns x and (noisy) y values.
    '''
    x = np.linspace(-5,5,data_points)
    y = A*x**2 + B 
    y_noisy = y + npr.normal(0.0,sigma,data_points)

    return x, y, y_noisy

x_data, y_ideal, y_data = my_model(A_user,B_user,sigma,data_points)


In [ ]:
# plot the data so we can see what's going on!
# Note: if you make A too big, it may be hard to see the errors.
plt.plot(x_data,y_ideal,'b--')
plt.errorbar(x_data, y_data, yerr=sigma, fmt='ro')


## MCMC fitting.  

The general idea behind MCMC fitting of our data (D) is that we will start from a guess at A$_{0}$ and B$_{0}$, and move in random directions in a way that on average gets us closer to the correct answer.  We keep track of the points that we've sampled over time, and use those to create a distribution.  The distribution shows us the likelihood of each (A,B) pair in our parameter space.  More specifically, we do the following:

1. Start with a guess for the model parameters $\theta = (A_{0}, B_{0})$, and calculate the "goodness of fit", or probability that this is the correct model, using some [likelihood function](https://en.wikipedia.org/wiki/Likelihood_function).  Since all we're going to do ultimately is look at the ratios of likelihoods from one step to another, we don't have to worry about how exactly we do this, so we will calculate this by calculating the sum-of-squares error $\chi^2$ at that point assuming some estimate of experimental error, and turn this into a likelihood function using a Gaussian: $P_{curr} = P(D,\theta_{curr}) = e^{-\chi_{curr}^2}$.  **Note that the likelihood function is larger when the error is smaller!**  (Because smaller difference between model and data means a better fit, hence the model is more likely to fit the data.  Hence, "likelihood."  Clever, eh?)

2. Take a step in a random direction in (A, B) space to get a new estimate of $\theta = (A_{new},B_{new})$, making sure to take a relatively small step so we don't overshoot the "real" answer.  We then calculate the likelihood function here, which we call $P_{prop}$.

3. We then calculate the quantity $P = P_{prop}/P_{curr} = e^{-\chi_{prop}^2 + \chi_{curr}^2}$ (in other words, the ratios of the likelihood functions for our proposed and current sets of (A,B) values). We then generate a random number R that's uniformly distributed in $[0,1)$.  If $R < P$, we "accept" the new (A, B) values.  Otherwise, we throw away the new values and try re-generating the new (A, B) values.  Note that if the new value of (A,B) is a better fit than the previous value, $P > 1$ and thus R will always be less than P.  However, even if the new (A, B) is a worse fit than the previous value there is still a possibility that we will move in that direction.

4. We do this for some number of tries which is defined either as fixed value or by some convergence criteria (we will discuss this in class).

We keep track of our accepted values of (A,B), which is the _trace_ of our Markov Chain _walker_.  This MC trace will give us an estimate of the probability distribution for our model parameters A and B, which we can then examine - the 'walker' will tend to stay in the region of good fit, but its wandering around parameter space gives us an idea of the range of likely values.

Note that we throw away a bunch of our initial guesses at (A,B), which are from the "burn-in" phase where we are migrating from our starting guess to a more reasonable range of parameters - look for the variable ``N_burn``.  The appropriate value for this variable varies, but ``N_burn = 1000`` is reasonable.

The code below presents an example of how this works for a simple model!

In [ ]:
## CONVENIENCE FUNCTIONS: these are needed for our fitting

def est_error(ydata,yguess,error):
    '''
    Takes in the observed data, our current step's estimated y-values 
    for the model, and our guess for the errors in the data.
    
    Returns sum-of-squares error assuming a 2-parameter model
    (the 2-parameter bit results in a -2 in the denominator)
    '''
    return ((ydata-yguess)**2/(2*error**2)).sum()/(ydata.size-2)


def est_model_vals(A,B,x):
    '''
    Given x-values and our MCMC code's estimates for A and B, this returns 
    estimated y values that we can compare to the actul data .
    '''
    return A*x**2 + B   


In [ ]:
## The actual MCMC code.

# Initial guesses for A and B
# Outcome should be relatively insensitive to these guesses!
Aold = 2.0
Bold = 2.0

# Range of step sizes that we take (linearly in -dA...dA and -dB...dB).
# This has to be relatively small, but if you make it too small you don't get
# Good sampling of the probability distribution function.
# Note that a better way to do this might be to sample in a Normal distribution instead of
# a linear distribution!
dA=dB=0.1

# Total number of points we're going to sample (should be at least 10^4, but more is generally 
# better unless you're in a real hurry)
Npts = 100000

# Number of points from the beginning that we want to throw away (should be 10^3)
N_burn = 1000

# Initial model values and 'error' for our starting position
y_old = est_model_vals(Aold,Bold,x_data)
err_old = est_error(y_data,y_old,estimated_error)

# These lists keep track of our Markov Chain and errors so we can get our PDF later.
Aguess = []
Bguess = []
errors = []

iter_count = 0

'''
This loop is where the magic happens.  We generate a new guess at (A,B) and compare
it to our original values using the chi^2 value.  If the new guess is better, we automatically
step in that direction.  If the new guess is worse, we still sometimes step in that direction if
a random number in the range [0,1) is less than the ratio of the chi^2 values for the old and new 
choices of (A,B). 
'''
for i in range(Npts):
    
    # Guess for our new point (A,B)
    # One may wish to replace npr.uniform() with npr.normal()
    Anew = Aold + npr.uniform(-dA,dA)
    Bnew = Bold + npr.uniform(-dB,dB)

    # model values based on the new A and B values
    ynew = est_model_vals(Anew,Bnew,x_data)
    
    # sum of squares value comparing data and model given our estimate
    # of the errors in the data.
    err_new = est_error(y_data, ynew,estimated_error)
    
    # acceptance probability - ratio of likelihoods for old and new data points.
    # note that the right hand side is equivalent to e^{-err_new} / e^{-err_old}
    p_accept = np.exp(-err_new+err_old)
    
    # if R < p_accept, we keep this point.  Otherwise, keep on moving!
    # remember if the new point is better than the old point this is always true 
    # since p_accept > 1 and npr.random() is uniform in the interval [0,1).  If the new
    # point is worse, p_accept < 1 so it sometimes happens anyway.
    if npr.random() < p_accept:
        Aold = Anew
        Bold = Bnew
        err_old = err_new
        
        # only keep track of points after our burn-in period.
        if iter_count > N_burn:
            Aguess.append(Aold)
            Bguess.append(Bold)
            errors.append(err_old)

    iter_count += 1


Now, we're going to take a look at the output of our models.  First, a raw plot of the trace of the Markov Chain.  In the plot below, the cyan circle is the user-chosen "real" value of (A,B) and the green square is the last point in the sample.  These are not necessarily near each other, since the MCMC 'walker' can travel around quite a bit and the last data point is simply the last step the walker took, not the best-fit model point.

In [ ]:
plt.plot(Aguess,Bguess,'b-',A_user,B_user,'co',markersize=10)
plt.plot(Aguess[-1],Bguess[-1],'gs',markersize=10)
plt.xlabel('A vals')
plt.ylabel('B vals')
plt.title('Markov Chain for estimate of (A,B)')

This is not nearly as informative as it could be - if you've taken enough steps to get a good sampling of the parameter space, it's very hard to understand.  So, now we're going to look at this MCMC trace in a different way, using a 2D histogram to see where the MCMC 'walker' spends most of its time as it wanders around our parameter space. We're going to look at the log of the 2D histogram.  The region where there are lots of points in the histogram represent more likely choices of (A,B) - i.e., a region of high confidence - and the blue/cyan/yellow regions represent less likely choices.  

In the plot below, the cyan circle is still the 'true' value, and the white diamond is in the histogram bin that the walker has visited the most, and is thus indicative of being a highly-favored value of the guess for (A,B).  Note that this particular estimate is somewhat noisy since the walker moves randomly, and there are better (less noisy, albeit somewhat more complex) ways of getting the "best" values).

How does the most likely region compare to the locations of the 'true' value and the "best" model value?

In [ ]:
from matplotlib.colors import LogNorm

cts,xbin,ybin,img = plt.hist2d(Aguess, Bguess, bins=60,norm=LogNorm())
plt.plot(A_user,B_user,'co',markersize=10)

# use np.argwhere() to find the bin(s) with the max counts
vals=np.argwhere(cts==cts.max())

# use those to guess our best values of A, B.  If there are more than one bins with the max
# number of counts, we're just using the first one we come to.  (This is not the best idea, but
# it makes the point)
Abest = xbin[vals[0,0]]
Bbest = ybin[vals[0,1]]
plt.plot(Abest,Bbest,'wD',markersize=10)

plt.colorbar()
plt.xlabel('A vals')
plt.ylabel('B vals')
plt.title('2D Histogram of walker positions')


We can also add contours onto the plot to see where the bulk of the points are.  Look at the pyplot [contour demo](http://matplotlib.org/examples/pylab_examples/contour_demo.html) for an example of how to work with this - basically, we're using the numpy array of output bins from the 2D histogram, and then adding contour lines on top of that.  For clarity, I'm showing the original histogram in greyscale with colored contours top of that.  The contour method choses the locations of the contours based on the counts in each bins, with the innermost contours enclosing the bins with the largest numbers of counts. (Note: these do NOT correspond to standard deviations/normal confidence intervals!)  We also add the user-set and best-fit model values as well, although this time we use the cyan circle for the user-set value and a magenta diamond for the 'best-fit' value (i.e., bin with the most counts in it).

In [ ]:
cts,xbin,ybin,img = plt.hist2d(Aguess, Bguess, bins=60,norm=LogNorm(),cmap='gray')

# plot contour.  Note that hist2d returns the bin edges so we have to average them to get the center.
# also, contour() and hist2d() treat the array differently so we need to transpose the cts array.
CS=plt.contour( 0.5*(xbin[1:]+xbin[:-1]),
                0.5*(ybin[1:]+ybin[1:]),
                cts.transpose(), 5, colors=('r','g','b','c','g'),
                linewidths=2)

# plot the user-chosen and best-fit points
plt.plot(A_user,B_user,'co',markersize=10)
plt.plot(Abest,Bbest,'mD',markersize=10)

We also will show the 1D histograms of the walker positions when collapsed into one dimension (i.e., just the values of A and, separately, the values of B).  The user-chosen 'true' value of each parameter is shown as the blue cyan line, and the 'best-fit' value is shown as the red line.  It's useful to note that the 1D histogram can be misleading - the peak of this histogram is not always the true best-fit value when you look at the fit in the maximum number of dimensions!

In [ ]:
foo=plt.hist(Aguess,bins=60)
plt.axvline(A_user, color='c', linestyle='dashed', linewidth=4)
plt.axvline(Abest, color='r', linestyle='dashed', linewidth=4)
plt.xlabel('A values')
plt.title('1D Histogram of A-values (from MCMC trace)')

In [ ]:
foo=plt.hist(Bguess,bins=60)
plt.axvline(B_user, color='c', linestyle='dashed', linewidth=4)
plt.axvline(Bbest, color='r', linestyle='dashed', linewidth=4)
plt.xlabel('B values')
plt.title('1D Histogram of B-values (from MCMC trace)')

Let's see how good of a job we've done in estimating the best-fit model!  In the plot below, the red data points with error bars represent the model and the errors we have assumed go along with those data points (which may or may not represent the *real* error in the measurements, as discussed above).  The blue line is the model that goes along with the best-guess values for A and B shown above (i.e., the white diamond in the previous plot).  The best-fit model should go through the data points rather convincingly.


In [ ]:
x_best_model, y_best_model, junk = my_model(Abest,Bbest,sigma=0.000001)
plt.plot(x_best_model,y_best_model,'b-',linewidth=3)
#plt.plot(x_data,y_data,'ro--',linewidth=1)
plt.errorbar(x_data, y_data, yerr=estimated_error, fmt='ro')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Data and best-fit model')

Finally, let's plot out the error in each step of the MC trace (i.e., the sum-of-squares error between the "real" data and the model at that step). Modify this plot to see a small chunk of it (say, a few hundred walker steps in the middle of the trace). You should see that the error starts out relatively large at the early steps and then gets small rather quickly, and then generally wanders around a moderate value.  The error can grow again because we allow the MCMC algorithm to randomly accept new choices of (A,B) that are actually a worse fit than our previous one, which is a method of sampling the parameter space and getting out of local minima in values! 

In [ ]:
plt.plot(errors)
plt.yscale('log')

# Some questions to answer

All of the questions below are asking for a **qualitative explanation** of what happens to the confidence region plot (the 2D histogram) when various model parameters are changed.

1.  What happens if you re-run the MCMC code several times without changing anything?  In other words, if you generate several different random instances, are the results generally similar, or do they vary widely?

2.  What happens to your confidence region as you change the errors in the model data?  In other words, how does the range of plausible (A,B) values change as you make ```sigma``` bigger or smaller?  If you wildly under- or over-estimate the error in the data points compared to reality (i.e., change ```sigma``` and ```estimated_error``` so that they are different relative to each other) what happens?

3.  What happens when you change your initial guess for (A,B)?  Try setting this to values that are quite close to the 'true' value, and then quite far away.

4.  What happens when you change the number of burn-in steps?  Try setting ```N_burn``` to 0 and to a much larger number than the default. 

**Put your answers here!**